# Fire Size Class Prediction

In [ ]:
# If a library is missing on your system, you can install it with the command :
# ! pip install <library name>
# For instance (remove # on the following line before executing)
# ! pip install pandas

If you face an error when running the previous command line, try :
- `! pip install --proxy=http://cias3basic.conti.de:8080/wpad.dat <library name>`
- you may also want to run the install command by deactivating the VPN connection, multiple people have reported facing difficulties downloading packages due to firewalls.

## 1. Data preparation

### 1.1 Data loading

The data we use is usually split into training data and test data. The training set contains a known output and the model learns on this data in order to be generalized to other data later on. We have the test dataset (or subset) in order to test our model’s prediction on this subset.

For this challenge, the division of the database into training and test datasets is not your responsibility. We provide you with the 2 separate CSV files.
CSV files can be easily read with the Pandas library. The resulting object is called a DataFrame.

For more info about the DataFrame object and functions, you can go here : https://www.tutorialspoint.com/python_pandas/python_pandas_dataframe.htm

In [1]:
import pandas as pd

train = pd.read_csv('train.csv')
train.head()

FileNotFoundError: ignored

In [ ]:
test = pd.read_csv('test.csv')
test.head()

As you can see, the test dataset does not contain the size_class, it will be your objective to predict it.

We can print the dataset sizes as well :

In [ ]:
print('Train set size:', train.shape)
print('Test set size:', test.shape)

### 1.2 Feature selection

Pay attention that our features are of different types - some of them are numeric, some are categorical, and some are even just strings, which normally should be handled in some specific way.

Indeed, categorical data are commonplace in many Data Science and Machine Learning problems but are usually more challenging to deal with than numerical data. In particular, many machine learning algorithms require that their input is numerical and therefore categorical features must be transformed into numerical features before we can use any of these algorithms.

In [ ]:
print(train.dtypes)

For getting started with a simple model, we will only use 2 numerical features (fire_location_latitude, fire_location_longitude) and add a few categorical features (fire_origin, true_cause, fire_type, fuel_type, weather_conditions_over_fire).

Now let's separate features and label variable. In most ML scenarios, X refers to the array of features and y is the variable to be predicted.

In [ ]:
X = train[['fire_location_latitude', 'fire_location_longitude', 'fire_origin', 'true_cause', 'fire_type', 'weather_conditions_over_fire', 'fuel_type']]
X.head()

In this challenge, the goal is to predict the __size class__ of the burned area, which consists in various classes :

In [ ]:
y = train['size_class']
y.unique()

This is a __supervised classification problem__ with 5 classes to predict :
- A class = 0 to 0.1 ha (1 hectare = 10000 sq meters = 2.5 acres)
- B class > 0.1 ha to 4.0 ha
- C class > 4.0 ha to 40.0 ha
- D class > 40.0 ha to 200 ha
- E class > 200 ha

To learn more about the different types of problems in Machine Learning, you can go here :
https://www.softwaretestinghelp.com/types-of-machine-learning-supervised-unsupervised/

### 1.3 Dealing with missing values

Let's check how many absent values do we have:

In [ ]:
null_value_stats = X.isnull().sum(axis=0)
null_value_stats[null_value_stats != 0]

As you can see, some features indeed have missing values.

Missing Value treatment becomes important since the data insights or the performance of your predictive model could be impacted if the missing values are not appropriately handled.

For now, let's just fill them with some arbitrary value so that the model would be able to easily distinguish between them and take it into account :

In [ ]:
X = X.fillna('Unknown') # str

### 1.4 Data Splitting

A validation dataset is a sample of data held back from training your model that is used to give an estimate of model performance while tuning model’s hyperparameters.

There is much confusion in applied ML about what a validation dataset is exactly and how it differs from a test dataset. Long story short :
- The validation dataset is predominately used to describe the evaluation of models when tuning hyperparameters and data preparation
- The test dataset is predominately used to describe the evaluation of a final tuned model when comparing it to other final models.

A good justification of this method can be found here : https://towardsdatascience.com/train-validation-and-test-sets-72cb40cba9e7

So let's split the train data into training and validation sets :

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.75, random_state=42)

### 1.5 Categorical feature encoding

As mentioned before, many machine learning algorithms require that their input is numerical and therefore categorical features must be transformed into numerical features before we can use any of these algorithms.

One of the most common ways to make this transformation is to __one-hot encode__ the categorical features, especially when there does not exist a natural ordering between the categories (e.g. a feature ‘City’ with names of cities such as ‘London’, ‘Lisbon’, ‘Berlin’, etc.). For each unique value of a feature (say, ‘London’) one column is created (say, ‘City_London’) where the value is 1 if for that instance the original feature takes that value and 0 otherwise.

A simple way to make this transformation is to use the category encoders library : http://contrib.scikit-learn.org/category_encoders/

In [ ]:
import category_encoders as ce

ohe = ce.OneHotEncoder(handle_unknown='value', use_cat_names=True)
X_train_ohe = ohe.fit_transform(X_train)
X_train_ohe.sample(5)

When we try to transform the test set, after having fitted the encoder to the training set, we may obtain a ValueError. This is because there are new, previously unseen unique values in the test set and the encoder doesn’t know how to handle these values. In order to use both the transformed training and test sets in machine learning algorithms, we need them to have the __same number of columns__.

This last problem can be solved by using the option `handle_unknown='value'` of the OneHotEncoder, which will ignore previously unseen values when transforming the test set and fill the resulting one-hot encoded columns for this feature with zeros.

In [ ]:
X_valid_ohe = ohe.transform(X_valid)
X_valid_ohe.sample(5)

As remarked previously, one-hot encoding is not the only possible way to encode categorical features and the category encoders library has several encoders which you should explore, as others might be more appropriate for different categorical features and machine learning problems.

## 2. Model Training

For this example, we will use the DecisionTreeClassifier from scikit-learn, which belongs to the family of supervised learning algorithms.

The goal of using a Decision Tree is to create a training model able to predict the class or value of the target variable by learning simple decision rules inferred from prior data (training data). To learn more about this algorithm :
https://www.mygreatlearning.com/blog/decision-tree-algorithm/

Note that we fixed the random_state value only to make the results reproducible for this notebook.

In [ ]:
from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier(random_state=42)
model.fit(X_train_ohe, y_train)

You can get a look at the feature importances and plot the decision tree as well.

### 2.1 Feature importance

Feature importance refers to techniques that assign a score to input features based on how useful they are at predicting a target variable :

In [ ]:
importance = pd.DataFrame()
importance['Feature'] = X_train_ohe.columns
importance['Importance'] = model.feature_importances_
importance.set_index('Feature', inplace=True)

importance.sort_values(by='Importance', ascending=False)

The importance of a feature is computed as the (normalized) total reduction of the criterion brought by that feature. It is also known as the Gini importance.

### 2.2 Plotting Tree

For clarity and in order to save time, we will only generate the first 3 levels of the tree. If `max_depth=None`, the tree is fully generated.

For this baseline model, it takes about 8 min to plot the full decision tree, but the result is horrible and practically unreadable !

In [ ]:
import matplotlib.pyplot as plt
from sklearn.tree import plot_tree

plt.figure(figsize=(20,15))
plot_tree(model, max_depth=3, feature_names=X_train_ohe.columns, filled=True, fontsize=10)
plt.show()

## 3. Model Validation

### 3.1 Predict on Validation dataset
All you have to do to get predictions on your validation set is :

In [ ]:
y_valid_pred = model.predict(X_valid_ohe)

### 3.2 Evaluation

Accuracy is one metric for evaluating classification models. Informally, accuracy is the fraction of predictions our model got right.

In [ ]:
from sklearn.metrics import accuracy_score

print('Validation accuracy score:', accuracy_score(y_valid, y_valid_pred))

A problem with this metric is that it doesn't take in consideration the fact that some classes are more represented than others, and unfortunately, it can actually lead to biased results.

In [ ]:
pd.DataFrame(y_valid, columns=['size_class'])['size_class'].value_counts()

Indeed, most machine learning algorithms assume data is equally distributed. So when we have a class imbalance, the machine learning classifier tends to be more biased towards the majority class, causing bad classification of the minority class.

To get a deeper understanding of this problem, you can go here : https://towardsdatascience.com/class-imbalance-a-classification-headache-1939297ff4a4

So in order to weight all classes equally, the evaluation metric for this challenge will be the __macro-average F1-score__ from scikit-learn :

In [ ]:
from sklearn.metrics import f1_score

print('Macro F1-score:', f1_score(y_valid, y_valid_pred, average='macro'))

Note that the F1-score with uniform weights is lower (0.29) than the overall accuracy (0.63) because it gives equal contribution to the predictive performance for the five classes, independent of their number of observations.

__Remember that this metric should be maximized.__

A confusion matrix is also a predictive analytics tool. Specifically, it is a table that displays and compares actual values with the model’s predicted values. Within the context of machine learning, a confusion matrix is utilized as a metric to analyze how a machine learning classifier performed on a dataset.

In [ ]:
from sklearn.metrics import plot_confusion_matrix

class_names = y_valid.unique()

# Plot non-normalized and normalized confusion matrices
titles_options = [("Confusion matrix, without normalization", None),
                  ("Normalized confusion matrix", 'true')]

for title, normalize in titles_options:
    disp = plot_confusion_matrix(model, X_valid_ohe, y_valid,
                                 labels=class_names,
                                 cmap=plt.cm.Blues,
                                 normalize=normalize)
    disp.ax_.set_title(title)

plt.show()

The classes that the model predicted correctly can be read on the diagonal on the matrix.

## 4. Make submission

### 4.1 Predict on Test dataset
Now we would re-train our model on all train data that we have.

In [ ]:
# Categorical Feature Encoding
X_ohe = ohe.fit_transform(X)

# Model Training
model.fit(X_ohe, y)

Don't forget to apply the same data transformations you used on your training set before applying the model on your test set:

In [ ]:
# Feature Selection
X_test = test[['fire_location_latitude', 'fire_location_longitude', 'fire_origin', 'true_cause', 'fire_type', 'weather_conditions_over_fire', 'fuel_type']]

# Missing values treatment
X_test = X_test.fillna('Unknown') # str
X_test.head()

In [ ]:
# Categorical Feature Encoding
X_test_ohe = ohe.transform(X_test)
X_test_ohe.sample(5)

And finally let's prepare the submission file:

In [ ]:
y_test_pred = pd.DataFrame()
y_test_pred['fire_number'] = test['fire_number']
y_test_pred['size_class'] = model.predict(X_test_ohe)

y_test_pred.head()

In [ ]:
print(y_test_pred.shape)

You should have 3969 rows in your submission file.

### 4.2 Push the prediction to CARINO platform

First you need to set the TOKEN value to the value you will find on the platform under the _Submissions_ tab :

In [ ]:
import math
import requests

TOKEN = None # Replace with 'your token' (please ensure value is between quotes)
comment = 'some comment' # Free text, visible by you only

def submit_prediction(df, TOKEN=None, sep=',', comment='', compression='gzip', **kwargs):
    if TOKEN is None:
        print("Please set TOKEN to the value provided in CARINO plarform under Submissions tab")
        return None
    URL='http://18.185.86.47/api/submissions'
    df.to_csv('temporary.dat', sep=sep, compression=compression, **kwargs)
    r = requests.post(URL, headers={'Authorization': 'Bearer {}'.format(TOKEN)},files={'datafile': open('temporary.dat', 'rb')},data={'comment':comment, 'compression': compression})
    if r.status_code == 429:
        raise Exception('Submissions are too close. Next submission is only allowed in {} minutes.'.format(int(math.ceil(int(r.headers['x-rate-limit-remaining']) / 1000.0 / 60.0))))
    if r.status_code != 200:
        raise Exception(r.text)

You can then submit your results as follows :

In [ ]:
submit_prediction(y_test_pred, TOKEN, sep=',', index=True, comment=comment)

You should now be able to check your score on the CARINO platform.

Note that you can save your predictions as a CSV file and upload it manually on the platform under the _Submissions_ tab later as well :

In [ ]:
y_test_pred.to_csv('submission.csv', index=False)

That's it ! Now you can play around with other models, add some feature engineering and try to improve your score ! :)